In [1]:
import torch
from llava.serve.cli import load_image
from llava.constants import IMAGE_TOKEN_INDEX
from llava.model.builder import load_pretrained_model
from llava.mm_utils import tokenizer_image_token, get_model_name_from_path, KeywordsStoppingCriteria
import json
from collections import defaultdict
from PIL import Image
import random

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


In [2]:
dataset_dir = 'E:/datasets'
dataset_name = 'DOTA_dataset_512'
coco_dir = f'{dataset_dir}/{dataset_name}'
coco_file = f'{coco_dir}/YOLO_coco_results_annots.json'

with open(coco_file, 'r') as f:
    coco_json = json.load(f)

image_id_to_annots = defaultdict(list)
for annot in coco_json['annotations']:
#    if annot['category_id'] in cats_to_use:
        image_id_to_annots[annot['image_id']].append(annot)

img_dict = random.choice(coco_json['images'])
image_path = img_dict['path']
img = Image.open(image_path).convert('RGB')
image_annotations = image_id_to_annots.get(img_dict['id'], [])

In [3]:


# Instantiate the model
model_path = "liuhaotian/llava-v1.5-7b"
model_base = None
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = get_model_name_from_path(model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(model_path, model_base, model_name, device=device)

# Process the image
image_file = image_path
image = load_image(image_file)
image_tensor = image_processor([image], return_tensors='pt')['pixel_values']
image_tensor = image_tensor.to(model.device, dtype=torch.float16)

# Generate image description based on input prompt
prompt = """A chat between a curious human and an artificial intelligence assistant. 
The assistant gives helpful, detailed, and polite answers to the human's questions. 
USER: <image> Can you describe this picture in detail?\nASSISTANT:"""

input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).to(model.device)
stopping_criteria = KeywordsStoppingCriteria(["</s>"], tokenizer, input_ids)

with torch.inference_mode():
    output_ids = model.generate(
        input_ids,
        images=image_tensor,
        do_sample=True,
        temperature=0.2,
        max_new_tokens=512,
        use_cache=True,
        stopping_criteria=[stopping_criteria])

output = tokenizer.decode(output_ids[0, input_ids.shape[1]:]).strip()
print("Output: ", output[:-4])

c:\Users\neals\anaconda3\envs\llava\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

c:\Users\neals\anaconda3\envs\llava\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\neals\.cache\huggingface\hub\models--liuhaotian--llava-v1.5-7b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for al

pytorch_model.bin.index.json:   0%|          | 0.00/27.1k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

c:\Users\neals\anaconda3\envs\llava\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

RuntimeError: "addmm_impl_cpu_" not implemented for 'Half'